In [1]:
gpu=4
CUDA_VISIBLE_DEVICES=gpu

In [2]:
import numpy as np
import pandas as pd

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler

import matplotlib.pyplot as plt
import time
import copy
from random import shuffle

import tqdm.notebook as tqdm

import sklearn
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.metrics import classification_report
from PIL import Image
import cv2

# import osa
import shutil

import seaborn as sns
import os
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torchvision
from sklearn.metrics import roc_curve,auc, precision_score,precision_recall_curve,recall_score,precision_recall_fscore_support,confusion_matrix
import numpy as np
from prettytable import PrettyTable
print(torch.cuda.is_available())
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
print(torch.cuda.get_device_properties(0).total_memory)
print(torch.cuda.memory_allocated())
gpu_id = gpu

True
11721506816
0


In [3]:
df=pd.read_csv("/DATA/chowdari1/DATA/dataset/stanford/CheXpert-v1.0-small/train.csv")

In [4]:
df.shape

(223414, 19)

In [5]:
df.isnull().values.sum()

2310361

In [6]:
df=df.fillna(-2)

In [7]:
df["Path"][3][-7:],df["Path"][4][-7:]

('ral.jpg', 'tal.jpg')

In [8]:
lcount=0
fcount=0
l=[]
for i in range(df.shape[0]):
    if(df['Path'][i][-7:]=='ral.jpg'):
        lcount+=1
        l.append(i)
    elif(df['Path'][i][-7:]=='tal.jpg'):
        fcount+=1
lcount, fcount,len(l)

(32387, 191027, 32387)

In [9]:
df=df.drop(l)
df=df.reset_index()
df.shape

(191027, 20)

In [10]:
lcount=0
fcount=0
l=[]
for i in range(df.shape[0]):
    if(df['Path'][i][-7:]=='ral.jpg'):
        lcount+=1
        l.append(i)
    elif(df['Path'][i][-7:]=='tal.jpg'):
        fcount+=1
lcount, fcount,len(l)

(0, 191027, 0)

In [11]:
s="No Finding"
df[s].unique()

array([ 1., -2.])

In [12]:
df.columns

Index(['index', 'Path', 'Sex', 'Age', 'Frontal/Lateral', 'AP/PA', 'No Finding',
       'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
       'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture',
       'Support Devices'],
      dtype='object')

In [13]:
# df["label"]
count=0
list=[]
for i in range(df.shape[0]):
    s=""
    # for j in df.columns[6:-2]:
    for j in df.columns[6:]:
        # print(j," ",s)
        if df[j][i]==1:
            # print("...")
            if(s!='' and (j=='Fracture' or j=='Support Devices')):
                s=s
            else:
                s=s+j
        # elif df[j][i]==-1:
        #     count+=1
    if( s=='Support Devices' or s=='Fracture' or s=='FractureSupport Devices'):
        s='s/f'
    list.append(s)
len(list),count

(191027, 0)

In [14]:
df["label"]=list

In [15]:
count=0
for i in range(df.shape[0]):
    if(df['No Finding'][i]==1 and df['label'][i]=='') :
        count+=1
count

0

In [16]:
count=0
for i in range(df.shape[0]):
    if(df['No Finding'][i]==1):
        count+=1
    elif(df['label'][i]=='s/f' and df['No Finding'][i]!=1):
        count+=1
    # if((df['No Finding'][i]==1 and df['label'][i]=='') or df['label'][i]=='s/f') :
        # count+=1
count

24171

In [17]:
len(list)

191027

In [18]:
df['label'].unique().shape

(658,)

In [19]:
df['No Finding'].value_counts()

-2.0    174053
 1.0     16974
Name: No Finding, dtype: int64

In [20]:
df['label'].value_counts()[:]

Lung OpacityPleural Effusion                                                                    19095
Lung Opacity                                                                                    18234
No Finding                                                                                      16974
Lung OpacityEdemaPleural Effusion                                                                8680
                                                                                                 7766
                                                                                                ...  
Enlarged CardiomediastinumCardiomegalyLung LesionPneumothorax                                       1
Enlarged CardiomediastinumCardiomegalyPneumothoraxPleural Effusion                                  1
Enlarged CardiomediastinumCardiomegalyLung LesionConsolidation                                      1
Enlarged CardiomediastinumLung LesionPleural EffusionPleural Other                

In [21]:
# df['tar']=np.empty(df.shape[0])

In [22]:
df['tar']=np.full(df.shape[0],-3)

In [23]:
df['tar'].value_counts()

-3    191027
Name: tar, dtype: int64

In [24]:
cnan=0
count0=0
count1=0
val=0
etc=0
for i in range(df.shape[0]):
    s= df['label'][i]
    if(df['No Finding'][i]==1 or s=='s/f'):
        count0+=1
        val=0
    elif(s=='' and df['No Finding'][i]!=1):
        cnan+=1
        val=-2
    elif(s!='' or s!='s/f'):
        val=1
        count1+=1
    else:
        etc+=1
    df['tar'][i]=val
count0,count1,cnan,etc


(24171, 159090, 7766, 0)

In [25]:
df['tar'].value_counts()

 1    159090
 0     24171
-2      7766
Name: tar, dtype: int64

In [26]:
for i in range(df.shape[0]):
    if(df['tar'][i]==-2):
        print(i)
        break

15


In [27]:
i

15

In [28]:
df.iloc[15]

index                                                                        20
Path                          CheXpert-v1.0-small/train/patient00011/study13...
Sex                                                                      Female
Age                                                                          22
Frontal/Lateral                                                         Frontal
AP/PA                                                                        PA
No Finding                                                                 -2.0
Enlarged Cardiomediastinum                                                  0.0
Cardiomegaly                                                               -2.0
Lung Opacity                                                               -2.0
Lung Lesion                                                                -2.0
Edema                                                                      -2.0
Consolidation                           

In [29]:
df['tar'].value_counts()

 1    159090
 0     24171
-2      7766
Name: tar, dtype: int64

In [30]:
l=[]
for i in range(df.shape[0]):
    if(df['tar'][i]==-2):
        l.append(i)
len(l)

7766

In [31]:
df=df.drop(l)


In [32]:
df=df.reset_index()

In [33]:
df['tar'].value_counts()

1    159090
0     24171
Name: tar, dtype: int64

In [34]:
df.head()

,level_0,index,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,...,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,label,tar
0,0,0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,-2.0,-2.0,...,-2.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,1.0,No Finding,0
1,1,1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,-2.0,-2.0,-1.0,...,-1.0,-2.0,-1.0,-2.0,-1.0,-2.0,1.0,-2.0,Lung Opacity,1
2,2,2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,-2.0,-2.0,-2.0,...,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,1.0,-2.0,Lung Opacity,1
3,3,4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,-2.0,-2.0,-2.0,...,-2.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,-2.0,Edema,1
4,4,5,CheXpert-v1.0-small/train/patient00004/study1/...,Female,20,Frontal,PA,1.0,0.0,-2.0,...,0.0,-2.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,No Finding,0


In [35]:
df.shape

(183261, 23)

In [36]:
df.iloc[15]

level_0                                                                      16
index                                                                        22
Path                          CheXpert-v1.0-small/train/patient00011/study1/...
Sex                                                                      Female
Age                                                                          19
Frontal/Lateral                                                         Frontal
AP/PA                                                                        AP
No Finding                                                                 -2.0
Enlarged Cardiomediastinum                                                 -2.0
Cardiomegaly                                                               -2.0
Lung Opacity                                                                1.0
Lung Lesion                                                                -2.0
Edema                                   

In [37]:
df.head()

,level_0,index,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,...,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,label,tar
0,0,0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,-2.0,-2.0,...,-2.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,1.0,No Finding,0
1,1,1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,-2.0,-2.0,-1.0,...,-1.0,-2.0,-1.0,-2.0,-1.0,-2.0,1.0,-2.0,Lung Opacity,1
2,2,2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,-2.0,-2.0,-2.0,...,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,1.0,-2.0,Lung Opacity,1
3,3,4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,-2.0,-2.0,-2.0,...,-2.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,-2.0,Edema,1
4,4,5,CheXpert-v1.0-small/train/patient00004/study1/...,Female,20,Frontal,PA,1.0,0.0,-2.0,...,0.0,-2.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,No Finding,0


In [38]:
df.shape

(183261, 23)

In [39]:
df['tar'].value_counts()

1    159090
0     24171
Name: tar, dtype: int64

In [40]:
# 24171
# nof=24171
df2=pd.DataFrame()
df2['path']=np.empty(df.shape[0])
df2['target']=np.zeros(df.shape[0])

In [45]:
df2['path']=df['Path']
df2['target']=df['tar']

In [46]:
df2.head()

,path,target
0,CheXpert-v1.0-small/train/patient00001/study1/...,0
1,CheXpert-v1.0-small/train/patient00002/study2/...,1
2,CheXpert-v1.0-small/train/patient00002/study1/...,1
3,CheXpert-v1.0-small/train/patient00003/study1/...,1
4,CheXpert-v1.0-small/train/patient00004/study1/...,0


In [50]:
df2['target'].value_counts()

1    159090
0     24171
Name: target, dtype: int64

In [ ]:
df2.to_csv('/DATA/chowdari1/DATA/dataset/stanford/wow.csv')

In [48]:
df2.iloc[0]

path      CheXpert-v1.0-small/train/patient00001/study1/...
target                                                    0
Name: 0, dtype: object

In [49]:
df.iloc[1]

level_0                                                                       1
index                                                                         1
Path                          CheXpert-v1.0-small/train/patient00002/study2/...
Sex                                                                      Female
Age                                                                          87
Frontal/Lateral                                                         Frontal
AP/PA                                                                        AP
No Finding                                                                 -2.0
Enlarged Cardiomediastinum                                                 -2.0
Cardiomegaly                                                               -1.0
Lung Opacity                                                                1.0
Lung Lesion                                                                -2.0
Edema                                   

In [57]:
# 24171
nof=24171
nodf=pd.DataFrame()
nodf['path']=np.empty(nof)
nodf['target']=np.zeros(nof)

In [37]:
df.shape[0]

183261

In [44]:
df.columns

Index(['level_0', 'index', 'Path', 'Sex', 'Age', 'Frontal/Lateral', 'AP/PA',
       'No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
       'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
       'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
       'Fracture', 'Support Devices', 'label', 'tar'],
      dtype='object')

In [50]:
df['tar'].value_counts()

1    159090
0     24171
Name: tar, dtype: int64

In [51]:
df.iloc[0]

level_0                                                                       0
index                                                                         0
Path                          CheXpert-v1.0-small/train/patient00001/study1/...
Sex                                                                      Female
Age                                                                          68
Frontal/Lateral                                                         Frontal
AP/PA                                                                        AP
No Finding                                                                  1.0
Enlarged Cardiomediastinum                                                 -2.0
Cardiomegaly                                                               -2.0
Lung Opacity                                                               -2.0
Lung Lesion                                                                -2.0
Edema                                   

In [58]:
count=0
for i in range(df.shape[0]):
    if(df['tar'][i]==0):
        nodf['path'][count]=df['Path'][i]
        # nodf['target'][i]=0
        count+=1
    if(count==24171):
        break
count, i

(24171, 183260)

In [80]:
# i

In [54]:
nodf['target'].value_counts()

0.000000e+00     24167
6.546276e-316        2
6.918623e-310        1
9.554044e-319        1
Name: target, dtype: int64

In [59]:
nodf.head()

,path,target
0,CheXpert-v1.0-small/train/patient00001/study1/...,0.0
1,CheXpert-v1.0-small/train/patient00004/study1/...,0.0
2,CheXpert-v1.0-small/train/patient00005/study1/...,0.0
3,CheXpert-v1.0-small/train/patient00006/study1/...,0.0
4,CheXpert-v1.0-small/train/patient00010/study1/...,0.0


In [60]:
path='/DATA/chowdari1/DATA/dataset/stanford'
nodf.to_csv(path+'/our/chexpert_nofinding.csv')